## The QMC and Quadrature Sampling Toy Examples

In [ ]:
import matplotlib.pyplot as plt
import importlib
%matplotlib inline
if importlib.util.find_spec("matplotlib_inline") is not None:
    import matplotlib_inline
    matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
else:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')

plt.ioff();

In [ ]:
# This cell is a sorry attempt at integrating the code onto google colab.
# Feel free to remove this cell if you are not using google colab.

import os, sys, sysconfig
from importlib.util import find_spec
is_colab = 'google.colab' in sys.modules
site_userpkg = sysconfig.get_paths()['purelib']
req_pkgscolab = ['tensorboardX', 'pyinstrument', 'chaospy']
if is_colab and any(find_spec(pkg) is None for pkg in req_pkgscolab):
    ! pip install {" ".join(req_pkgscolab)}
if is_colab and not os.path.exists('/content/btspinn/'):
    ! cd /content && git clone https://github.com/ehsansaleh/btspinn.git
    ! cd /content/btspinn && pip install -e . && rm -rf *.egg-info
if is_colab and (find_spec('bspinn') is None):
    ! ln -s /content/btspinn/bspinn {site_userpkg}/bspinn
elif is_colab:
    ! [ -L {site_userpkg}/bspinn ] && rm {site_userpkg}/bspinn
if is_colab and os.path.exists('/content/btspinn/notebook'):
    os.chdir('/content/btspinn/notebook')

In [ ]:
import numpy as np
import torch
import json
import time
import scipy
import shutil
import socket
import random
import chaospy
import pathlib
import fnmatch
import datetime
import resource
import itertools
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorboardX
import psutil
from pyinstrument import Profiler
from torch import nn
from copy import deepcopy
from itertools import chain
from scipy.special import gamma
from os.path import exists, isdir
from collections import defaultdict
from collections import OrderedDict as odict
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from bspinn.io_utils import DataWriter
from bspinn.io_utils import get_git_commit
from bspinn.io_utils import preproc_cfgdict
from bspinn.io_utils import hie2deep, deep2hie
from bspinn.io_utils import drop_unqcols

from bspinn.tch_utils import isscalar
from bspinn.tch_utils import EMA
from bspinn.tch_utils import BatchRNG
from bspinn.tch_utils import bffnn
from bspinn.tch_utils import profmem

from bspinn.io_cfg import configs_dir
from bspinn.io_cfg import results_dir
from bspinn.io_cfg import storage_dir

# Spherical Coordinates

The spherical coordinate system in $d$ dimensions relates $[x_1, \cdots, x_d]$ to $[r, \phi_1, \phi_2, \cdots, \phi_{d-1}]$ such that

$$x_1 = r \cos(\phi_1),$$
$$x_2 = r \sin(\phi_1)\cos(\phi_2),$$
$$x_2 = r \sin(\phi_1)\sin(\phi_2)\cos(\phi_3),$$
$$\ldots$$
$$x_{d-1} = r \sin(\phi_1)\sin(\phi_2)\cdots \cos(\phi_{d-1}),$$
$$x_d = r \sin(\phi_1)\sin(\phi_2)\cdots \sin(\phi_{d-1}).$$

We also have 
$$\phi_1, \cdots, \phi_{d-2} \in [0, \pi],$$ 
$$\phi_{d-1} \in [0, 2\pi).$$

The Jacobian determinent is therefore

$$\det([\frac{\partial x_i}{\partial(r, \phi_j)}]_{i,j}) = r^{d-1} \sin^{d-2}(\phi_1) \sin^{d-3}(\phi_2) \cdots \sin(\phi_{d-2}).$$

In other words, we have

$$ \text{d}^{d}(V) = r^{d-1} \sin^{d-2}(\phi_1) \sin^{d-3}(\phi_2) \cdots \sin(\phi_{d-2}) \text{d}r \text{d}\phi_1 \cdots \text{d} \phi_{d-1}.$$


# Uniform Sphere Sampling
Define $F_n$ such that 

$$F_n(u) = \frac{1}{\int_{0}^{\pi} \sin^{n}(\phi) \text{d}{\phi}} \int_{0}^{\pi u} \sin^{n}(\phi) \text{d}{\phi}.$$

Note that $F_n(0) = 0$ and $F_n(1)=1$, and that $F_n$ is basically a CDF for a $\sin^{n}$-like PDF.

To sample points uniformly from the sphere of the $d$-dimensional unit ball, here is one process:

1. Sample $u = [u_1, u_2, \cdots, u_{d-1}]$ uniformly from $[0, 1]^{d-1}$.

2. Construct an inverse map for $F_n$ where $1 \leq n \leq {d-2}$. This can be done using a lookup table and `torch.searchsorted` for instance.

3. Compute the following:

$$\phi_1 = \pi \cdot F_{d-2}^{-1}(u_1),$$
$$\phi_2 = \pi \cdot F_{d-3}^{-1}(u_2),$$
$$\ldots$$
$$\phi_{d-2} = \pi \cdot F_{1}^{-1}(u_{d-2}),$$
$$\phi_{d-1} = 2 \pi \cdot u_{d-1}.$$

4. Translate $[r, \phi_1, \phi_2, \cdots, \phi_{d-1}]$ to the cartesian system $[x_1, \cdots, x_d]$.

Resources: 

1. https://en.wikipedia.org/wiki/N-sphere

2. http://corysimon.github.io/articles/uniformdistn-on-sphere/

In [ ]:
class Cube2Sphere:
    def __init__(self, n_cdfint, dim, tch_device, tch_dtype):
        self.n_cdfint = n_cdfint
        self.dim = dim
        self.tch_device = tch_device
        self.tch_dtype = tch_dtype
        self.cdftab = self.get_cdftab(n_cdfint, dim)
        
    def tch_exlinspace(self, start, end, n):
        assert n >= 1
        a = torch.linspace(start, end, n+1,
                           device=self.tch_device,
                           dtype=self.tch_dtype)[:-1]
        b = a + 0.5 * (end - a[-1])
        return b
        
    def get_cdftab(self, n_cdfint, dim):
        r"""
        This function computes the necessary CDF functions $F_n$ for $1\leq n \leq dim-1$ such that 

            $$F_n(u) = \frac{1}{\int_{0}^{\pi} \sin^{n}(\phi) \text{d}{\phi}} \int_{0}^{\pi u} \sin^{n}(\phi) \text{d}{\phi}.$$

        Note that $F_n(0) = 0$ and $F_n(1)=1$, and that $F_n$ is basically a CDF for a $\sin^{n}$-like PDF.
        
        Parameters
        ----------
        n_cdfint: (int) The number of points for CDF integration and table lookup.

        dim: (int) The dimension of the space. The unit sphere should be a 
            `dim-1` dimensional manifold.
        """
        # Step 0: Defining the inverse CDF mapping
        thunif1d = self.tch_exlinspace(0.0, np.pi, n_cdfint)
        assert thunif1d.shape == (n_cdfint,)

        thunif = thunif1d.reshape(1, n_cdfint).expand(dim-1, n_cdfint)
        assert thunif.shape == (dim-1, n_cdfint)

        sinpow = torch.arange(dim-2, -1, -1, device=self.tch_device).reshape(dim-1, 1)
        assert sinpow.shape == (dim-1, 1)

        thsinpow = torch.sin(thunif) ** sinpow
        assert thsinpow.shape == (dim-1, n_cdfint)

        cdftab = thsinpow.cumsum(dim=-1)
        cdftab = cdftab / cdftab[:, -1:]
        assert cdftab.shape == (dim-1, n_cdfint)
        
        return cdftab
        
    def __call__(self, unifs):
        """
        Takes a set of uniform random values in the $[0, 1]^{dim-1}$ cube, and transforms it 
        to the points on the surface of the $d$-dimensional unit-ball.
        
        The transformation is designed in a way such that uniform inputs in the rectangle lead
        to uniform points on the surface of the unit-ball.
        
        Parameters
        ----------
        unifs: (torch.tensor) An input tensor with all values between 0 and 1. This input can 
            be batched. The shape of this tensor must end with `dim-1`.
            
            Example:
                `dim = 5`
                `unifs = torch.rand(100, 10, 25, 4)`
            
        Output
        ------
        x_cart: (torch.tensor) The output points on the unit sphere in the cartesian system.
            This will have the same batch dimensions as the input.
            
            Example:
                `dim = 5`
                `unifs = torch.rand(100, 10, 25, 4)`
                `assert x_cart.shape == (100, 10, 25, 5)`
        """
        dim, cdftab, n_cdfint = self.dim, self.cdftab, self.n_cdfint
        tch_dtype, tch_device = self.tch_dtype, self.tch_device
        assert unifs.shape[-1] == dim-1
        
        u_mbdims = unifs.shape[:-1]
        n_samp = np.prod(u_mbdims)
        
        # Step 2: Applying the inverse CDF mapping
        unifs_T = unifs.reshape(n_samp, dim-1).T.contiguous()
        assert unifs_T.shape == (dim-1, n_samp)

        cdfranks_T = torch.searchsorted(cdftab, unifs_T)
        assert cdfranks_T.shape == (dim-1, n_samp)

        cdfranks = cdfranks_T.T.reshape(n_samp, dim-1)
        assert cdfranks.shape == (n_samp, dim-1)

        cdfinvu = (cdfranks / n_cdfint).to(unifs.dtype)
        assert cdfinvu.shape == (n_samp, dim-1)

        # Step 3: Scaling the uniform values to the phi ranges
        #   Note: $\phi_{d-1}$ should be in the $[0, 2\pi]$ range, unlike 
        #         the $[0, \pi]$ range for the rest of the coordinates.
        phi_scale = torch.tensor([np.pi]*(dim-2) + [2*np.pi]).to(
            dtype=tch_dtype, device=tch_device).reshape(1, dim-1)
        assert phi_scale.shape == (1, dim-1)

        phi = cdfinvu * phi_scale
        assert phi.shape == (n_samp, dim-1)

        # Step 4: Translating to cartesian coordinates
        phi_cos_ = torch.cos(phi)
        assert phi_cos_.shape == (n_samp, dim-1)

        phi_cos = torch.cat([phi_cos_, torch.ones(n_samp, 1, 
            dtype=tch_dtype, device=tch_device)], dim=-1)
        assert phi_cos.shape == (n_samp, dim)

        phi_sin = torch.sin(phi)
        assert phi_sin.shape == (n_samp, dim-1)

        phi_sincumprod_ = phi_sin.cumprod(dim=-1)
        assert phi_sincumprod_.shape == (n_samp, dim-1)

        phi_sincumprod = torch.cat([torch.ones(n_samp, 1, dtype=tch_dtype, 
            device=tch_device), phi_sincumprod_], dim=-1)
        assert phi_sincumprod.shape == (n_samp, dim)

        x_cart_ = phi_sincumprod * phi_cos
        assert x_cart_.shape == (n_samp, dim)

        # Making sure the points lie on the unit sphere
        assert x_cart_.square().sum(dim=-1).allclose(torch.ones(1, dtype=tch_dtype, 
            device=tch_device))

        x_cart = x_cart_.reshape(*u_mbdims, dim)
        assert x_cart.shape == (*u_mbdims, dim)
        
        return x_cart
    


In [ ]:
class SphereSampler:
    def __init__(self, batch_rng):
        self.tch_dtype = batch_rng.dtype
        self.tch_device = batch_rng.device
        self.batch_rng = batch_rng

    def np_exlinspace(self, start, end, n):
        assert n >= 1
        a = np.linspace(start, end, n, endpoint=False)
        b = a + 0.5 * (end - a[-1])
        return b

    def tch_exlinspace(self, start, end, n):
        assert n >= 1
        a = torch.linspace(start, end, n+1,
                           device=self.tch_device,
                           dtype=self.tch_dtype)[:-1]
        b = a + 0.5 * (end - a[-1])
        return b

    def __call__(self, volumes, n, trnsfrm_params, samp_params, do_randrots=True, do_shflpts=True):
        # volumes -> dictionary
        assert volumes['type'] == 'ball'
        centers = volumes['centers']
        radii = volumes['radii']
        n_bch, n_v, d = centers.shape
        use_np = not torch.is_tensor(centers)
        assert centers.shape == (n_bch, n_v, d)
        assert radii.shape == (n_bch, n_v)
        assert not (use_np) or (self.batch_rng.lib == 'numpy')
        assert use_np or (self.batch_rng.device == centers.device)
        assert use_np or (self.batch_rng.dtype == centers.dtype)
        assert self.batch_rng.shape == (n_bch,)
        exlinspace = self.np_exlinspace if use_np else self.tch_exlinspace
        meshgrid = np.meshgrid if use_np else torch.meshgrid
        sin = np.sin if use_np else torch.sin
        cos = np.cos if use_np else torch.cos
        arccos = np.arccos if use_np else torch.arccos
        matmul = np.matmul if use_np else torch.matmul
        
        # Phase 0: Input arguments processing
        
        # Taking shallow copies
        trnsfrm_params, samp_params = dict(trnsfrm_params), dict(samp_params)
        trnsfrm_mthd = trnsfrm_params.pop('dstr')
        if trnsfrm_mthd == 'cube2sphr':
            cube2sphr = trnsfrm_params.pop('cube2sphr')
            rv_dim = d - 1
        elif trnsfrm_mthd == 'normscale':
            rv_dim = d
        else:
            raise RuntimeError('Not implemented')
        assert len(trnsfrm_params) == 0, f'unknown params: {trnsfrm_params}'
        
        samp_mthd = samp_params.pop('dstr')
        if samp_mthd == 'quad':
            quad_x = samp_params.pop('x')
            assert quad_x.shape == (n, rv_dim)
            quad_w = samp_params.pop('w')
            assert quad_w.shape == (n,)
            n_bch_, n_v_ = 1, 1
        elif samp_mthd == 'qmc':
            qmc_x = samp_params.pop('x')
            assert qmc_x.shape == (n, rv_dim)
            n_bch_, n_v_ = 1, 1
        elif samp_mthd == 'rng':
            n_bch_, n_v_ = n_bch, n_v
        elif samp_mthd == 'grid':
            n_bch_, n_v_ = 1, 1
        else:
            raise RuntimeError('Not implemented')
        assert len(samp_params) == 0, f'unknown params: {samp_params}'
        en_randrots = do_randrots and (samp_mthd in ('grid', 'quad', 'qmc'))
        en_shflpts = do_shflpts and (samp_mthd in ('quad', 'qmc', 'grid'))
        
        # Phase 1: Creating the `norms`, `unifs`, and `weights` variables.
        #          We either create or sample these variables, or process 
        #          the input arguments to create them.
        if (samp_mthd == 'grid') and (trnsfrm_mthd == 'normscale'):
            n_droot = int(np.round(n ** (1.0 / d)))
            assert n == (n_droot ** d), f'N={n} should have an integer {d} root (n_droot={n_droot})'
            u1d = exlinspace(0, 1, n_droot)
            assert u1d.shape == (n_droot,)
            n1d = torch.erfinv(2 * u1d - 1) * np.sqrt(2)
            norms = torch.cartesian_prod(*([n1d] * d)).reshape(1, 1, n, d)
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'grid') and (trnsfrm_mthd == 'cube2sphr'):
            n_droot = int(np.round(n ** (1.0 / (d-1))))
            assert n == (n_droot ** (d-1)), f'N={n} should have an integer {d-1} root'
            u1d = exlinspace(0, 1, n_droot)
            assert u1d.shape == (n_droot,)
            unifs = torch.cartesian_prod(*([u1d] * (d-1))).reshape(1, 1, n, d-1)
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'rng') and (trnsfrm_mthd == 'normscale'):
            norms = self.batch_rng.normal((n_bch, n_v, n, d))
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'rng') and (trnsfrm_mthd == 'cube2sphr'):
            unifs = self.batch_rng.uniform((n_bch, n_v, n, d-1))
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'quad') and (trnsfrm_mthd == 'normscale'):
            norms = quad_x.reshape(1, 1, n, d)
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = quad_w.reshape(1, 1, n).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd  == 'quad') and (trnsfrm_mthd == 'cube2sphr'):            
            unifs = quad_x.reshape(1, 1, n, d-1)
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = quad_w.reshape(1, 1, n).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'qmc') and (trnsfrm_mthd == 'normscale'):
            # norms = torch.erfinv(2 * qmc_x.reshape(1, 1, n, d) - 1) * np.sqrt(2)
            norms = qmc_x.reshape(1, 1, n, d)
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'qmc') and (trnsfrm_mthd == 'cube2sphr'):
            unifs = qmc_x.reshape(1, 1, n, d-1)
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        else:
            raise RuntimeError('Not implemented yet!')

        # End of Phase 1. At this point, we should have the following 
        # satisfied under all conditions.
        assert weights.shape == (n_bch, n_v, n)
        if (trnsfrm_mthd == 'cube2sphr'):
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            assert (unifs >= 0.0).all()
            assert (unifs <= 1.0).all()
            rands = unifs
        else:
            assert norms.shape == (n_bch_, n_v_, n, d)
            assert (norms.square().sum(dim=-1) > 0).all()
            rands = norms
            
        # Phase 2: Transforming `norms`/`unifs` to points on the unit-sphere.
        #          Inputs: `norms` and `unifs`.
        #          Outputs: `x_tilde_`
        if trnsfrm_mthd == 'normscale':
            norms_l2 = torch.sqrt(torch.square(norms).sum(dim=-1))
            x_tilde_ = norms / norms_l2.reshape(n_bch_, n_v_, n, 1)
            assert x_tilde_.shape == (n_bch_, n_v_, n, d)
        elif trnsfrm_mthd == 'cube2sphr':
            if cube2sphr is not None:
                assert cube2sphr is not None
                x_tilde_ = cube2sphr(unifs)
                assert x_tilde_.shape == (n_bch_, n_v_, n, d) 
            elif (cube2sphr is None) and (d in [2, 3]):
                if d == 2:
                    theta_2d = unifs * (2 * np.pi)
                    assert theta_2d.shape == (n_bch_, n_v_, n, 1)
                    x_tilde_list = [cos(theta_2d), sin(theta_2d)]
                elif d == 3:
                    u1_2d, u2_2d = unifs[..., :1], unifs[..., 1:]
                    assert u1_2d.shape == (n_bch_, n_v_, n, 1)
                    assert u2_2d.shape == (n_bch_, n_v_, n, 1)
                    phi_2d = arccos(1- 2 * u2_2d)
                    theta_2d = u1_2d * (2 * np.pi)
                    x_tilde_list = [sin(phi_2d) * cos(theta_2d),
                                    sin(phi_2d) * sin(theta_2d), 
                                    cos(phi_2d)]
                else:
                    raise RuntimeError('Not implemented yet!')
                if use_np:
                    x_tilde_ = np.concatenate(x_tilde_list, axis=-1)
                else:
                    x_tilde_ = torch.cat(x_tilde_list, dim=-1)
                assert x_tilde_.shape == (n_bch_, n_v_, n, d)
            else:
                raise RuntimeError('Not implemented yet!')
        else:
            raise RuntimeError('Not implemented yet!')
        
        # End of Phase 2. At this point, we should have the following
        # satisfied under all conditions
        assert x_tilde_.shape == (n_bch_, n_v_, n, d)
        
        # Phase 3: Final touches: random rotations, shuffling, constant calculations, etc.
        x_tilde = x_tilde_.expand(n_bch, n_v, n, d)
        assert x_tilde.shape == (n_bch, n_v, n, d)
        
        if en_shflpts:
            rngunifs = self.batch_rng.uniform((n_bch, n_v, n))
            assert rngunifs.shape == (n_bch, n_v, n)
            randperm3d = rngunifs.argsort(dim=-1)
            assert randperm3d.shape == (n_bch, n_v, n)
            randperm4d = randperm3d.reshape(n_bch, n_v, n, 1)
            assert randperm4d.shape == (n_bch, n_v, n, 1)
            
            x_tilde_shfld = torch.take_along_dim(x_tilde, randperm4d, dim=-2)
            assert x_tilde_shfld.shape == (n_bch, n_v, n, d)
            weights_shfld = torch.take_along_dim(weights, randperm3d, dim=-1)
            assert weights_shfld.shape == (n_bch, n_v, n)
            rands_shfld = torch.take_along_dim(rands.expand(n_bch, n_v, n, rv_dim), randperm4d, dim=-2)
            assert rands_shfld.shape == (n_bch, n_v, n, rv_dim)
        else:
            x_tilde_shfld = x_tilde
            assert x_tilde_shfld.shape == (n_bch, n_v, n, d)
            weights_shfld = weights
            assert weights_shfld.shape == (n_bch, n_v, n)
            rands_shfld = rands.expand(n_bch, n_v, n, rv_dim)
            assert rands_shfld.shape == (n_bch, n_v, n, rv_dim)

        if en_randrots:
            rot_mats = self.batch_rng.so_n((n_bch, n_v, d, d))
            assert rot_mats.shape == (n_bch, n_v, d, d)
            
        if en_randrots:
            x_tilde_rot = matmul(x_tilde_shfld, rot_mats)
        else:
            x_tilde_rot = x_tilde_shfld
        assert x_tilde_rot.shape == (n_bch, n_v, n, d)

        points = x_tilde_rot * \
            radii.reshape(n_bch, n_v, 1, 1) + centers.reshape(n_bch, n_v, 1, d)
        assert points.shape == (n_bch, n_v, n, d)

        if use_np:
            x_tilde_bc = np.broadcast_to(x_tilde_shfld, (n_bch, n_v, n, d))
        else:
            x_tilde_bc = x_tilde_shfld.expand(n_bch, n_v, n, d)

        if en_randrots:
            rot_x_tilde = matmul(x_tilde_bc, rot_mats)
        else:
            rot_x_tilde = x_tilde_bc
        assert rot_x_tilde.shape == (n_bch, n_v, n, d)

        cst = (2*(np.pi**(d/2))) / gamma(d/2)
        csts = cst * (radii**(d-1))
        assert csts.shape == (n_bch, n_v)

        ret_dict = dict(points=points, normals=rot_x_tilde, weights=weights_shfld, areas=csts, rands=rands_shfld)
        return ret_dict



In [ ]:
ex_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
ex_tchdevice = torch.device(ex_device)
ex_tchdtype = torch.double

n_bch = 5
rng = BatchRNG(shape=(n_bch,), lib='torch', device=ex_tchdevice, dtype=ex_tchdtype,
               unif_cache_cols=10000, norm_cache_cols=50000)
rng.seed(np.broadcast_to(12345+np.arange(n_bch), rng.shape))
ex_dim, n_v = 3, 10

vols = dict(type='ball', centers=torch.zeros(n_bch, n_v, ex_dim, device=ex_tchdevice, dtype=ex_tchdtype),
            radii=torch.ones(n_bch, n_v, device=ex_tchdevice, dtype=ex_tchdtype))

sphsampler_3d = SphereSampler(batch_rng=rng)

In [ ]:
cube2sphr = Cube2Sphere(n_cdfint=10000, dim=3, tch_device=ex_tchdevice, tch_dtype=ex_tchdtype)

all_sphrcfgs = []
for trnsfrm_mthd in ['cube2sphr', 'normscale']:
    ex_tparams = dict(dstr=trnsfrm_mthd)
    if trnsfrm_mthd == 'cube2sphr':
        ex_tparams['cube2sphr'] = cube2sphr
        
    quad_sphrcfgs = []
    rv_dim = ex_dim-1 if trnsfrm_mthd == 'cube2sphr' else ex_dim
    if trnsfrm_mthd == 'cube2sphr':
        cpy_dist = chaospy.Iid(chaospy.Uniform(0, 1), rv_dim)
    else:
        cpy_dist = chaospy.Iid(chaospy.Normal(0, 1), rv_dim)
    for rule, sparse, rcuralg in itertools.product(["radau", "gaussian", "lobatto", "legendre", 'c', 
        'f1', 'f2', 'z16', 'z18', 'z22', 'z24', 'kronrod','leja', 'n', 'patterson'], 
        [False, True], ["stieltjes"]):
        ex_quadx_np, ex_quadw_np = chaospy.generate_quadrature(
            order=2, dist=cpy_dist, rule=rule, recurrence_algorithm=rcuralg,
            sparse=sparse)
        
        if trnsfrm_mthd == 'cube2sphr':
            assert np.isclose(min(0.0, ex_quadx_np.min()), 0.0)
            assert np.isclose(max(1.0, ex_quadx_np.max()), 1.0)
            ex_quadx_np = np.clip(ex_quadx_np, 0.0, 1.0)
        elif trnsfrm_mthd == 'normscale':
            ex_nz_ind = np.square(ex_quadx_np).sum(axis=0) >= 1e-25
            ex_quadx_np = ex_quadx_np[:, ex_nz_ind]
            ex_quadw_np = ex_quadw_np[ex_nz_ind]
        else:
            raise ValueError('not implemented yet')
        ex_n = ex_quadw_np.size
        ex_quadx = torch.from_numpy(ex_quadx_np.T).to(device=ex_tchdevice, dtype=ex_tchdtype)
        assert ex_quadx.shape == (ex_n, rv_dim)
        ex_quadw = torch.from_numpy(ex_quadw_np).to(device=ex_tchdevice, dtype=ex_tchdtype) * ex_n
        assert ex_quadw.shape == (ex_n,)
        ex_sparams = {'dstr': 'quad', 'x': ex_quadx, 'w': ex_quadw}
        
        ex_ttl = f'{rule.capitalize()}, {"Sprase" if sparse else "Dense"}'
        sphrcfg = {'trnsfrm_params': ex_tparams, 'samp_params': ex_sparams,  
            'n': ex_n, 'title': ex_ttl}
        quad_sphrcfgs.append(sphrcfg)
    ax_ttl = f'Quadrature Abscissas '
    if trnsfrm_mthd == 'cube2sphr':
        ax_ttl += f'from {rv_dim}D Unit Cube, Transformed to 3D Unit Sphere'
    else:
        ax_ttl += f'from {rv_dim}D Normal Distributtion, Normalized to 3D Unit Sphere'
    all_sphrcfgs.append([ax_ttl, quad_sphrcfgs])
    
    qmc_sphrcfgs = []
    if trnsfrm_mthd == 'cube2sphr':
        cpy_dist = chaospy.Iid(chaospy.Uniform(0, 1), ex_dim-1)
    else:
        cpy_dist = chaospy.Iid(chaospy.Normal(0, 1), ex_dim)
    cpy_j = chaospy.J(cpy_dist)
    ex_n = 64
    for rule in ['additive_recursion', 'halton', 'hammersley', 'korobov', 'sobol', 'latin_hypercube']:
        ex_qmcx_np = cpy_j.sample(ex_n, rule=rule)
        
        if trnsfrm_mthd == 'cube2sphr':
            assert np.isclose(min(0.0, ex_qmcx_np.min()), 0.0)
            assert np.isclose(max(1.0, ex_qmcx_np.max()), 1.0)
            ex_qmcx_np = np.clip(ex_qmcx_np, 0.0, 1.0)
        elif trnsfrm_mthd == 'normscale':
            ex_qmcx_np = ex_qmcx_np + 1e-12 # Important to avoid all zeros that cannot be normalized
            assert (np.square(ex_qmcx_np).sum(axis=0) > 0.0).all()
        else:
            raise ValueError('not implemented yet')
        
        ex_qmcx = torch.from_numpy(ex_qmcx_np.T).to(device=ex_tchdevice, dtype=ex_tchdtype)
        assert ex_qmcx.shape == (ex_n, rv_dim)
        
        ex_sparams = {'dstr': 'qmc', 'x': ex_qmcx}
        ex_ttl = rule.replace('_recursion', ' r.').replace("_", " ").title()
        sphrcfg = {'trnsfrm_params': ex_tparams, 'samp_params': ex_sparams,  
            'n': ex_n, 'title': ex_ttl}
        qmc_sphrcfgs.append(sphrcfg)
    
    ax_ttl = f'Quasi-Random Points '
    if trnsfrm_mthd == 'cube2sphr':
        ax_ttl += f'Sampled from {rv_dim}D Unit Cube, Transformed to 3D Unit Sphere'
    else:
        ax_ttl += f'Sampled from {rv_dim}D Normal Distribution, Normalized to 3D Unit Sphere'
    all_sphrcfgs.append([ax_ttl, qmc_sphrcfgs])
        
    rg_sphrcfgs = []
    rg_sphrcfgs.append({'trnsfrm_params': ex_tparams, 'samp_params': dict(dstr='grid'),  
        'n': 64, 'title': 'Grid'})

    rg_sphrcfgs.append({'trnsfrm_params': ex_tparams, 'samp_params': dict(dstr='rng'),  
        'n': 64, 'title': 'RNG'})
    ax_ttl = f'Simple {rv_dim}D '
    if trnsfrm_mthd == 'cube2sphr':
        ax_ttl += f'Uniform Vars, Transformed to 3D Unit Sphere'
    else:
        ax_ttl += f'Normal Vars, Normalized to 3D Unit Sphere'
    all_sphrcfgs.append([ax_ttl, rg_sphrcfgs])

In [ ]:
for fig_ttl, fig_sphrcfgs in all_sphrcfgs:
    for ex_sphrcfg in fig_sphrcfgs:
        ex_sphsamps = sphsampler_3d(volumes=vols, n=ex_sphrcfg['n'], 
            trnsfrm_params=ex_sphrcfg['trnsfrm_params'], 
            samp_params=ex_sphrcfg['samp_params'],
            do_randrots=False)
        ex_sphrcfg['sphsamps'] = ex_sphsamps
        assert not ex_sphsamps['points'].isnan().any()
        assert not ex_sphsamps['weights'].isnan().any()
        assert not ex_sphsamps['normals'].isnan().any()
        assert not ex_sphsamps['areas'].isnan().any()
        assert not ex_sphsamps['rands'].isnan().any()
        assert ex_sphsamps['points'].square().sum(-1).allclose(
            torch.ones_like(ex_sphsamps['points'][...,0]))
        

In [ ]:
all_figs = []
np_random = np.random.RandomState(seed=12345)
c_blue = sns.color_palette('dark', 12)[0]
for fig_ttl, fig_sphrcfgs in all_sphrcfgs:
    n_ax = len(fig_sphrcfgs)
    n_cols = min(4, n_ax)
    n_rows = int(np.ceil(n_ax / n_cols))
    fig = plt.figure(figsize=(3.0*n_cols, 3.0*n_rows), dpi=100)
    axes = [fig.add_subplot(n_rows, n_cols, i+1, projection='3d') for i in range(n_ax)]

    for ax, ex_sphrcfg in zip(axes, fig_sphrcfgs):
        ex_sphsamps = ex_sphrcfg['sphsamps']
        ex_points = ex_sphsamps['points'][0, 0].detach().cpu().numpy()
        
        ex_points = ex_points + np_random.randn(*ex_points.shape) * 0.0
        ax.scatter(ex_points[:, 2], ex_points[:, 1], ex_points[:, 0], marker='o', c=c_blue, alpha=0.5, s=12)
        ax.set_xlim3d(-1, 1)
        ax.set_ylim3d(-1, 1)
        ax.set_zlim3d(-1, 1)

        ax.set_box_aspect((2, 2, 2))
        ax.set_xticks([-1.0, 0.0, 1.0])
        ax.set_xticklabels([-1.0, 0.0, 1.0])
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_zticks([])
        ax.set_zticklabels([])
        ax.grid(False)
        ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax.quiver(-1.0, -1.0, -1.0, 2.4, 0.0, 0.0, color='k', arrow_length_ratio=0.1)
        ax.quiver(-1.0, -1.0, -1.0, 0.0, 2.4, 0.0, color='k', arrow_length_ratio=0.1)
        ax.quiver(-1.0, -1.0, -1.0, 0.0, 0.0, 2.4, color='k', arrow_length_ratio=0.1)
        ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
        ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
        ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
        
        ax.tick_params(axis='both', which='major', pad=-5)
        
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x = 1.0 * np.cos(u)*np.sin(v)
        y = 1.0 * np.sin(u)*np.sin(v)
        z = 1.0 * np.cos(v)
        ax.plot_wireframe(x, y, z, color="black", lw=0.1)

        ttl = ax.set_title(f'{ex_sphrcfg["title"]}, N={ex_sphrcfg["n"]}', y=0.92)
    fig.suptitle(fig_ttl, y={1: 0.93, 2: 0.93, 3: 0.93, 4: 0.92, 5: 0.91, 6: 0.91, 7: 0.90, 8: 0.89}[n_rows])
    fig.subplots_adjust(wspace={1: 0.0, 2: 0.0, 3: -0.1, 4: -0.25}.get(n_cols, 0.0), hspace=0.05)
    all_figs.append(fig)

In [ ]:
workdir, figcnt = f'./22_quadquas', 1
! mkdir -p {workdir}
for fig in all_figs:
    fig.savefig(f'{workdir}/{figcnt:02d}_points.pdf', bbox_inches="tight")
    figcnt += 1

## Quadrature Examples

The following figure shows points sampled from the 2D uniform transformed to 3D unit sphere.

This was done through applying an initial CDF inversion then a spherical coordinate converstion

In [ ]:
all_figs[0]

The following figure shows points sampled from the 3D Normal distribution, then L2 normalized to be placed on the unit sphere.

*Note*: ChaosPy took care of generating quadrature points from the normal distribution. In other words, we did not apply a CDF inversion to convert the uniform variables into normal ones.

In [ ]:
all_figs[3]

## Quasi-Random Examples

The following figure shows points sampled from the 2D uniform transformed to 3D unit sphere.

This was done through applying an initial CDF inversion then a spherical coordinate converstion

In [ ]:
all_figs[1]

The following figure shows points sampled from the 3D Normal distribution, then L2 normalized to be placed on the unit sphere.

*Note*: ChaosPy took care of generating Quasi-random numbers from the normal distribution. In other words, we did not apply a CDF inversion to convert the uniform variables into normal ones.

In [ ]:
all_figs[4]

## Simple Grid and RNG Examples

The following figure shows points sampled from the 2D uniform transformed to 3D unit sphere.

This was done through applying an initial CDF inversion then a spherical coordinate converstion

In [ ]:
all_figs[2]

The following figure shows points sampled from the 3D Normal distribution, then L2 normalized to be placed on the unit sphere.

The grid points were first transformed to quasi-normal random variables by applying an inverse Gaussian CDF.

In [ ]:
all_figs[5]

# The Underlying Abscissas before any Spherical or Normalization Transformations

In [ ]:
all_rfigs = []
np_random = np.random.RandomState(seed=12345)
c_blue = sns.color_palette('dark', 12)[0]
for fig_ttl, fig_sphrcfgs in all_sphrcfgs:
    n_ax = len(fig_sphrcfgs)
    n_cols = min(4, n_ax)
    n_rows = int(np.ceil(n_ax / n_cols))
    plt_dim = 3 if fig_sphrcfgs[0]['trnsfrm_params']['dstr'] == 'normscale' else 2
    if plt_dim == 3:
        fig = plt.figure(figsize=(3.0*n_cols, 3.0*n_rows), dpi=100)
        axes = [fig.add_subplot(n_rows, n_cols, i+1, projection='3d') for i in range(n_ax)]
    else:
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.0*n_cols, 3.0*n_rows), sharex=True, sharey=True, dpi=100)
        axes = np.array(axes).ravel()
    [ax.remove() for ax in axes[n_ax:]]

    for ax, ex_sphrcfg in zip(axes, fig_sphrcfgs):
        ex_sphsamps = ex_sphrcfg['sphsamps']
        assert not ex_sphsamps['rands'].isnan().any()
        ex_rands = ex_sphsamps['rands'][0, 0].detach().cpu().numpy()
        ex_rands = ex_rands + np_random.randn(*ex_rands.shape) * 0.0001
        if plt_dim == 3:
            ax.scatter(ex_rands[:, 0], ex_rands[:, 1], ex_rands[:, 2], marker='o', s=8)
            ttl = ax.set_title(f'{ex_sphrcfg["title"]}, N={ex_sphrcfg["n"]}', y=.95)
            
            # ax.invert_yaxis()
            ax.set_xticks([-3.0, 0.0, 3.0])
            ax.set_xticklabels([-3.0, 0.0, 3.0])
            ax.set_yticks([])
            ax.set_yticklabels([])
            ax.set_zticks([])
            ax.set_zticklabels([])
            ax.grid(False)
            ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
            ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
            ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
            ax.quiver(-3.0, -3.0, -3.0, 6.5, 0.0, 0.0, color='k', arrow_length_ratio=0.1)
            ax.quiver(-3.0, -3.0, -3.0, 0.0, 6.5, 0.0, color='k', arrow_length_ratio=0.1)
            ax.quiver(-3.0, -3.0, -3.0, 0.0, 0.0, 6.5, color='k', arrow_length_ratio=0.1)
            ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
            ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
            ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
            
            ax.tick_params(axis='both', which='major', pad=-5)
            
            u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
            x = 1.0 * np.cos(u)*np.sin(v)
            y = 1.0 * np.sin(u)*np.sin(v)
            z = 1.0 * np.cos(v)
            ax.plot_wireframe(x, y, z, color="black", lw=0.1)
            
            ax.set_xlim3d(-3, 3)
            ax.set_ylim3d(-3, 3)
            ax.set_zlim3d(-3, 3)
            ax.set_box_aspect((2, 2, 2))
            
            fig.subplots_adjust(wspace={1: 0.0, 2: 0.0, 3: -0.1, 4: -0.29}.get(n_cols, 0.0), hspace=0.05)
        else:
            ax.scatter(ex_rands[:, 0], ex_rands[:, 1], marker='*', s=40, alpha=0.5, c=c_blue)
            ax.set_xlim(-0.05, 1.05)
            ax.set_ylim(-0.05, 1.05)
            ttl = ax.set_title(f'{ex_sphrcfg["title"]}, N={ex_sphrcfg["n"]}')
    
    fig_yttl = {1: 0.93, 2: 0.96, 3: 0.93, 4: 0.92, 5: 0.91, 6: 0.91, 7: 0.90, 8: 0.905}[n_rows]
    fig_yttl = 1.05 if n_ax == 2 else fig_yttl
    fig.suptitle(fig_ttl.split(',')[0], y=fig_yttl)
    all_rfigs.append(fig)

In [ ]:
for fig in all_rfigs:
    fig.savefig(f'{workdir}/{figcnt:02d}_abscissas.pdf', bbox_inches="tight")
    figcnt += 1

## Quadrature Examples

The following figure shows points sampled from the 2D unit cube.

This was done through applying an initial CDF inversion then a spherical coordinate converstion

In [ ]:
all_rfigs[0]

The following figure shows points sampled from the 3D Normal distribution, which would be later L2 normalized to be placed on the unit sphere.

*Note*: ChaosPy took care of generating quadrature points from the normal distribution. In other words, we did not apply a CDF inversion to convert the uniform variables into normal ones.

In [ ]:
all_rfigs[3]

## Quasi-Random Examples

The following figure shows points sampled from the 2D uniform variables.

This was done through applying an initial CDF inversion then a spherical coordinate converstion

In [ ]:
all_rfigs[1]

The following figure shows points sampled from the 3D Normal distribution, before bing L2 normalized to be placed on the unit sphere.

*Note*: ChaosPy took care of generating Quasi-random numbers from the normal distribution. In other words, we did not apply a CDF inversion to convert the uniform variables into normal ones.

In [ ]:
all_rfigs[4]

## Simple Grid and RNG Examples

The following figure shows points sampled from the 2D unit cube.

In [ ]:
all_rfigs[2]

The following figure shows points sampled from the 3D Normal distribution, before being L2 normalized to be placed on the unit sphere.

The grid points were then transformed to quasi-normal random variables by applying an inverse Gaussian CDF.

In [ ]:
all_rfigs[5]

# Sample Size Growth

In [ ]:
cart_vars = odict()
cart_vars['order'] = [2, 3]
cart_vars['dim'] = [2, 3, 4, 5, 6, 7, 8, 9, 10]
cart_vars['rule'] = ["radau", "gaussian", "lobatto", "legendre"]
cart_vars['rcuralg'] = ['chebyshev', 'stieltjes']
cart_vars['sparse'] = [False, True]
cart_vars['trnsfrm_mthd'] = ['normscale']

df_rowdicts = []
for vals in itertools.product(*cart_vars.values()):
    cfg = {key: val for key, val in zip(cart_vars.keys(), vals)}
    print('.', end='')
    if len(df_rowdicts) % 50 == 49:
        print('')

    if cfg['trnsfrm_mthd'] == 'cube2sphr':
        cpy_dist = chaospy.Iid(chaospy.Uniform(0, 1), cfg['dim']-1)
    else:
        cpy_dist = chaospy.Iid(chaospy.Normal(0, 1), cfg['dim'])
    ex_quadx_np, ex_quadw_np = chaospy.generate_quadrature(
        order=cfg['order'], dist=cpy_dist, rule=cfg['rule'], 
        recurrence_algorithm=cfg['rcuralg'], sparse=cfg['sparse'])    
    cfg['n'] = ex_quadw_np.size
    df_rowdicts.append(cfg)
df = pd.DataFrame(df_rowdicts)

In [ ]:
pltdf = df.copy(deep=True)
pltdf['sparse'] = ['Sparse' if sprs else 'Dense' for sprs in pltdf['sparse']]
pltdf['order'] = [f'Order={x}' for x in pltdf['order']]
pltdf['trnsfrm_mthd'] = ['C2S' if trnsfrm_mthd == 'cube2sphr' else 'Norm' 
                         for trnsfrm_mthd in pltdf['trnsfrm_mthd']]
pltdf['rule'] = pltdf['rule'].str.capitalize()
pltdf = pltdf.rename(columns={'dim': 'Dimension'})

hue_col = 'rule'
xcol = 'Dimension'
ycol = 'n'

pltdf = drop_unqcols(pltdf)
grp_cols = [col for col in pltdf.columns if col not in (hue_col, xcol, ycol)]
dfgrps = list(pltdf.groupby(grp_cols))

n_ax = len(dfgrps)
n_cols = min(4, n_ax)
n_rows = int(np.ceil(n_ax / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.0*n_cols, 3.0*n_rows), 
    sharex=True, sharey=True, dpi=100)
axes = np.array(axes).ravel()
[ax.remove() for ax in axes[n_ax:]]

sns_pal = sns.color_palette('bright', 12)
hue2clr = {col: clr for col, clr in zip(pltdf[hue_col].unique().tolist(), sns_pal)}

for ax_idx, (grp_vals, ax_df) in enumerate(dfgrps):
    ax = axes[ax_idx]
    ax_row, ax_col = ax_idx // n_cols, ax_idx % n_cols 
    for hue_val, hue_df in ax_df.groupby(hue_col):
        ax.plot(hue_df[xcol], hue_df[ycol], c=hue2clr[hue_val], lw=2, marker='*', label=hue_val)
    if ax_row == (n_rows - 1):
        ax.set_xlabel(xcol.title())
    if ax_col == 0:
        ax.set_ylabel(ycol.title())
    ax.set_yscale('log', base=10)
    ax.set_title(', '.join([str(x).title() for x in grp_vals]))
    ax.set_xticks([2, 3, 4, 5, 6, 7, 8, 9, 10])
    ax.set_xticklabels([2, 3, 4, 5, 6, 7, 8, 9, 10])

ax.legend()
fig

In [ ]:
fig.savefig(f'{workdir}/{figcnt:02d}_n_vs_dim.pdf', bbox_inches="tight")
figcnt += 1

## Writing down the exact sample sizes

In [ ]:
cart_vars = odict()
cart_vars['order'] = [2]
cart_vars['trnsfrm_mthd'] = ['cube2sphr', 'normscale']
cart_vars['rcuralg'] = ["stieltjes", 'chebyshev']
cart_vars['dim'] = [2, 3, 4, 5, 6, 7, 8, 9, 10]
cart_vars['rule'] = ['leja', 'lobatto', 'radau', 'legendre',
    'clenshaw_curtis', 'newton_cotes', 'gaussian', 'fejer_2',
    'genz_keister_16','patterson', 'genz_keister_18', 'genz_keister_22',
    'genz_keister_24', 'kronrod','fejer_1']
cart_vars['sparse'] = [True]

df_rowdicts = []
for vals in itertools.product(*cart_vars.values()):
    cfg = {key: val for key, val in zip(cart_vars.keys(), vals)}
    print('.', end='')
    if len(df_rowdicts) % 50 == 49:
        print('')

    if cfg['trnsfrm_mthd'] == 'cube2sphr':
        cpy_dist = chaospy.Iid(chaospy.Uniform(0, 1), cfg['dim']-1)
    else:
        cpy_dist = chaospy.Iid(chaospy.Normal(0, 1), cfg['dim'])
    ex_quadx_np, ex_quadw_np = chaospy.generate_quadrature(
        order=cfg['order'], dist=cpy_dist, rule=cfg['rule'], 
        recurrence_algorithm=cfg['rcuralg'], sparse=cfg['sparse'])    
    cfg['n'] = ex_quadw_np.size
    df_rowdicts.append(cfg)
df = pd.DataFrame(df_rowdicts)

In [ ]:
printdf = df[(df['order'] == 2)]
all_lines = []
for grp_vals, grp_df in printdf.groupby(['sparse', 'rule', 'trnsfrm_mthd', 'rcuralg'], sort=False):
    n_list = (grp_df.sort_values(by='dim')['n']-1).tolist()
    sprs, rule, trnsfrm_mthd, rcuralg = grp_vals
    line = f'{rule}, {trnsfrm_mthd}, {rcuralg}: {n_list}'
    all_lines.append(line)
    
splt_lines = [line.split(',') for line in all_lines]
n_rows, n_cols = len(splt_lines), len(splt_lines[0])
out_spltlines = [[] for _ in range(n_rows)]
for j in range(n_cols):
    col_len = max(len(line[j]) for line in splt_lines)
    for i in range(n_rows):
        line = splt_lines[i]
        out_spltlines[i].append(line[j] + ',' + ' ' * (col_len - len(line[j])))
out_lines = [''.join(line) for line in out_spltlines]
max_outlen = max(len(line.split(']')[0]) for line in out_lines)
for line in out_lines:
    line_ = line.split(']')[0]
    line__ = line_ + ' ' * (max_outlen - len(line_)) + ']'
    print(line__)